In [1]:
# Module Importations
from scipy.stats import reciprocal
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import keras
import numpy as np
import time

Using TensorFlow backend.


In [2]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.models import keras_helpers

In [3]:
# Load dataset
original_dataset = load_data.load_motor_data()

In [4]:
# Split data into training / evaluation sets
training_set, evaluation_set = split_data.split_train_test(original_dataset, 0.8)

In [5]:
# Drop profile id data column
training_set = training_set.drop("profile_id", axis = 1)

In [6]:
# Create rotor target
rotor_training_data = training_set.drop("pm", axis = 1)
rotor_label_data = training_set["pm"].copy()

In [7]:
# Create train and test arrays
X_train, X_test, y_train, y_test = train_test_split(rotor_training_data, rotor_label_data, test_size = 0.2, random_state = 0)

In [8]:
# Multilayer Perceptron (Target - Rotor Temperature)

# Clear existing models
keras.backend.clear_session()

# Parameter distribution
param_distribs = { 
    "n_hidden": [1, 2], 
    "n_neurons": np.arange(1, 100), 
    "learning_rate": reciprocal(3e-4, 3e-2)
    }

# Build model
wrapped_model = keras_helpers.wrap_model()

# Initialise Random Search
rnd_search_cv = RandomizedSearchCV(wrapped_model, param_distribs, n_iter = 10, cv = 3)

# Name model file
model_id = keras_helpers.name_model()
filepath_full = keras_helpers.make_save_string(model_id)

# Set save & earlystop callbacks
earlystop_cb = keras.callbacks.EarlyStopping(patience = 5)
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

# Train model
rnd_search_cv.fit( X_train, y_train, epochs = 5, validation_data =(X_test, y_test), callbacks =[checkpoint_cb, earlystop_cb])

Train on 106460 samples, validate on 39923 samples
Epoch 1/5
106460/106460 [==============================] - 6s 60us/step - loss: 0.4473 - val_loss: 0.3127
Epoch 2/5
106460/106460 [==============================] - 7s 64us/step - loss: 0.2861 - val_loss: 0.2782
Epoch 3/5
106460/106460 [==============================] - 6s 61us/step - loss: 0.2669 - val_loss: 0.2670
Epoch 4/5
106460/106460 [==============================] - 5s 47us/step - loss: 0.2583 - val_loss: 0.2595
Epoch 5/5
53231/53231 [==============================] - 1s 18us/step
Train on 106461 samples, validate on 39923 samples
Epoch 1/5
106461/106461 [==============================] - 6s 52us/step - loss: 0.4701 - val_loss: 0.3111
Epoch 2/5
106461/106461 [==============================] - 6s 53us/step - loss: 0.2899 - val_loss: 0.2839
Epoch 3/5
106461/106461 [==============================] - 7s 63us/step - loss: 0.2712 - val_loss: 0.2683
Epoch 4/5
106461/106461 [==============================] - 5s 47us/step - loss: 0.2584

KeyboardInterrupt: 

In [9]:
# Multilayer Perceptron (Target - Rotor Temperature)
model_name = "YC_0001"


# Clear existing models
keras.backend.clear_session()

# Create model
mlp_model = keras_helpers.create_multilayer_perceptron(X_train.shape[1])

# Train model
mlp_model = keras_helpers.train_multilayer_perceptron(mlp_model, X_train, X_test, y_train, y_test)

# Evaluate model
y_pred = mlp_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared = False)
mae = mean_absolute_error(y_test, y_pred)

print(model_name, "MSE: ", str(rmse))
print(model_name, "MAE", str(mae))

AttributeError: module 'src.models.keras_helpers' has no attribute 'create_multilayer_perceptron'

In [10]:
# Multilayer Perceptron (Target - Rotor Temperature)
model_name = "YC_0002"

# Clear existing models
keras.backend.clear_session()

# Create model
mlp_model = keras_helpers.create_multilayer_perceptron(X_train.shape[1])

# Train model
mlp_model = keras_helpers.train_multilayer_perceptron(mlp_model, X_train, X_test, y_train, y_test)

# Evaluate model
y_pred = mlp_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared = False)
mae = mean_absolute_error(y_test, y_pred)

print(model_name, "MSE: ", str(rmse))
print(model_name, "MAE", str(mae))

AttributeError: module 'src.models.keras_helpers' has no attribute 'create_multilayer_perceptron'

In [11]:
# Multilayer Perceptron (Target - Rotor Temperature)
model_name = "YC_0003"

# Clear existing models
keras.backend.clear_session()

# Create model
mlp_model = keras_helpers.create_multilayer_perceptron(X_train.shape[1])

# Train model
mlp_model = keras_helpers.train_multilayer_perceptron(mlp_model, X_train, X_test, y_train, y_test)

# Evaluate model
y_pred = mlp_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared = False)
mae = mean_absolute_error(y_test, y_pred)

print(model_name, "MSE: ", str(rmse))
print(model_name, "MAE", str(mae))

AttributeError: module 'src.models.keras_helpers' has no attribute 'create_multilayer_perceptron'

In [12]:
# Multilayer Perceptron (Target - Rotor Temperature)
model_name = "YC_0004"

# Clear existing models
keras.backend.clear_session()

# Create model
mlp_model = keras_helpers.create_multilayer_perceptron(X_train.shape[1])

# Train model
mlp_model = keras_helpers.train_multilayer_perceptron(mlp_model, X_train, X_test, y_train, y_test)

# Evaluate model
y_pred = mlp_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared = False)
mae = mean_absolute_error(y_test, y_pred)

print(model_name, "MSE: ", str(rmse))
print(model_name, "MAE", str(mae))

AttributeError: module 'src.models.keras_helpers' has no attribute 'create_multilayer_perceptron'

In [13]:
# Multilayer Perceptron (Target - Rotor Temperature)
model_name = "YC_0005"

# Clear existing models
keras.backend.clear_session()

# Create model
mlp_model = keras_helpers.create_multilayer_perceptron(X_train.shape[1])

# Train model
mlp_model = keras_helpers.train_multilayer_perceptron(mlp_model, X_train, X_test, y_train, y_test)

# Evaluate model
y_pred = mlp_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared = False)
mae = mean_absolute_error(y_test, y_pred)

print(model_name, "MSE: ", str(rmse))
print(model_name, "MAE", str(mae))

AttributeError: module 'src.models.keras_helpers' has no attribute 'create_multilayer_perceptron'

In [14]:
full_name = "YC_0005_SEQ_11-100-60-30-1_ReLU_MSE_Adam"

keras_helpers.save_sequential_model(mlp_model, full_name)

AttributeError: module 'src.models.keras_helpers' has no attribute 'save_sequential_model'